## Projeto 04 ⭐
Modelo de Classificação

#### Caracteristicas (features, x)
* mileage_per_year

* model_year

* price


#### Classificação (target, y)

**sold**

* 1 = VENDEU

* 0 = NAO VENDEU

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [8]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,preco,vendido,idade_do_modelo,km_por_ano
0,0,30941.02,1,18,35085.22134
1,1,40557.96,1,20,12622.05362
2,2,89627.50,0,12,11440.79806
3,3,95276.14,0,3,43167.32682
4,4,117384.68,1,4,12770.11290


In [9]:
dados = dados.drop(columns = ["Unnamed: 0"], axis=1)

In [10]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [12]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [22]:
SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size= 0.25, stratify = y)
print("Treino: {}, Teste: {}".format(len(treino_x), len(teste_x)))

Treino: 7500, Teste: 2500


In [14]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acuracia do dummy stratified %.2f%%" % acuracia)

A acuracia do dummy stratified 50.96%


## Arvore de Decisao


In [15]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 71.92%


## Cross-validation

In [16]:
from sklearn.model_selection import cross_validate

In [29]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy CROSS VALIDATION = 3 [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao)*100))

Accuracy CROSS VALIDATION = 3 [74.99 76.57]


In [41]:
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy CROSS VALIDATION = 5 [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao)*100))

Accuracy CROSS VALIDATION = 5 [75.21 76.35]


## Aleatoriedade no cross validate

In [42]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy %.2f" % (media * 100))
    print("Accuracy CROSS VALIDATION = 10 [%.2f %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao)*100))

In [43]:
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

imprime_resultados(results)

Accuracy 75.78
Accuracy CROSS VALIDATION = 10 [74.37 77.19]


In [46]:
# shuffle - aleatoriedade
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

imprime_resultados(results)

Accuracy 75.76
Accuracy CROSS VALIDATION = 10 [73.26 78.26]


## Estratificação com validação cruzada


### Simular situação horrível de azar

Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

In [48]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]

In [49]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)

imprime_resultados(results)

Accuracy 57.84
Accuracy CROSS VALIDATION = 10 [34.29 81.39]


In [51]:
# shuffle - aleatoriedade
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)

imprime_resultados(results)

Accuracy 75.78
Accuracy CROSS VALIDATION = 10 [72.30 79.26]


In [52]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)

imprime_resultados(results)

Accuracy 75.78
Accuracy CROSS VALIDATION = 10 [73.55 78.01]


## Dados agrupaveis
Gerando dados aleatórios de modelo de carro para simulacao de agrupamento ao usar o estimador

In [58]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [59]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

In [61]:
abs(dados.modelo_aleatorio.min())

1

In [64]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [65]:
abs(dados.modelo_aleatorio.min())

1

In [71]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,18,18
1,40557.96,1,20,12622.05362,24,24
2,89627.50,0,12,11440.79806,14,14
3,95276.14,0,3,43167.32682,6,6
4,117384.68,1,4,12770.11290,5,5


In [74]:
dados = dados.drop(columns = ["modelo_aleatorio"], axis=1)

In [75]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [69]:
dados.modelo_aleatorio.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [72]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [73]:
dados.modelo.value_counts()

,count
modelo,
20,901
19,798
18,771
21,723
17,709
16,668
14,621
22,575
15,573


## Validação cruzada usando grupos

Testando a validacao cruzada com GroupKFold

In [76]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)

imprime_resultados(results)

Accuracy 75.78
Accuracy CROSS VALIDATION = 10 [73.83 77.73]


## Cross validation com StandardScaler

In [79]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [81]:
SEED = 301
np.random.seed(SEED)


scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)


modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)


acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 74.40%


In [84]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)

imprime_resultados(results)

Accuracy 77.27
Accuracy CROSS VALIDATION = 10 [74.35 80.20]


In [88]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [89]:
x_azar_escalado

array([[ 0.3636103 , -0.39591706,  0.24651152],
       [ 0.79214444, -0.18257669,  0.03346607],
       [ 0.72310419,  1.09746554,  1.66435195],
       ...,
       [ 0.17822399, -0.60925743, -0.73405662],
       [ 0.2248779 ,  0.45744443, -0.39741686],
       [-1.34266351,  0.88412517,  1.52099053]])

In [91]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)

imprime_resultados(results)

Accuracy 76.70
Accuracy CROSS VALIDATION = 10 [74.30 79.10]


In [95]:
from sklearn.pipeline import Pipeline

SEED = 301

scaler = StandardScaler()
modelo = SVC()
pipeline = Pipeline([("transformacao", scaler), ("estimador", modelo)])

np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy 76.68
Accuracy CROSS VALIDATION = 10 [74.28 79.08]
